#  One way Pipeline

Flow: 
```
USER prompt -> Blender CodeGen -> Render -> VLM Verifier
```

## 0  Setup and dependencies

In [135]:
# ↳ Run once per environment
!source spatial-env/bin/activate
# !pip install -r requirements.txt

## 1  Scene‑graph schema & helper classes

In [136]:
import os
from openai import OpenAI
import json
from dotenv import load_dotenv

import time
# import uuid 

import subprocess
import pathlib
import cloudinary
import cloudinary.uploader

from cloudinary.utils import cloudinary_url
import os
from loguru import logger

load_dotenv()

True

In [137]:
result = {
    "id": 1,
    "test_type": "multi-relation",
    "prompt": "",
    "image_url": "",
    "test_questions": {
        "question_1": {
            "question": "",
            "answer": ""
        },
        "question_2": {
            "question": "",
            "answer": ""
        },
        "question_3": {
            "question": "",
            "answer": ""
        },
        "question_4": {
            "question": "",
            "answer": ""
        }
    }
}

## 2  Blender code generation.

In [138]:
MODEL = 'gpt-4'

OpenAPIClient = OpenAI(api_key="")

def llm_to_scene(prompt: str) -> dict:

    system_message = """
                        You are a 3D spatial planning assistant. 
                        Generate blender code for scene with room layouts with proper furniture placement, considering spatial relationships, accessibility, and design principles.
                        Only return the blender code, no other text.
        """
    
    try:
        response = OpenAPIClient.chat.completions.create(
            model=MODEL,
            messages=[
                {'role':'system', 'content': system_message},
                {'role':'user', 'content': prompt}
            ],
        )
        print(response.choices[0].message.content)
    
            
        logger.info(f"blender code generated successfully ...")
        print(response.choices[0].message.content)
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"Error in llm_to_scene: {str(e)}")
        raise

## 3  Constraint solver → absolute coordinates

## 4  CodeGen → Blender .py script

## 5  Headless Blender render

In [139]:
cloudinary.config(
    cloud_name=os.getenv('CLOUDINARY_CLOUD_NAME'),
    api_key=os.getenv('CLOUDINARY_API_KEY'),
    api_secret=os.getenv('CLOUDINARY_API_SECRET')
)

def render_and_upload_image(pyfile: str, image_outfile: str = 'assets/one_step_renders/scene.png',
                           cloudinary_folder: str = 'one_step_renders', prompt: str = ''):
    """
    Uploading rendered images (PNG, JPG, etc.)
    
    Args:
        pyfile: Path to the Python script for Blender
        image_outfile: Output path for the rendered image
        cloudinary_folder: Folder name in Cloudinary
    
    Returns:
        dict: Cloudinary upload response
    """
    logger.info(f"Rendering and uploading image to Cloudinary ...")
    blender_bin = '/Applications/Blender.app/Contents/MacOS/Blender'
    
    output_path = pathlib.Path(image_outfile)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    cmd = [blender_bin, '-b', '--python', pyfile]
    logger.info(f"Running Blender: {' '.join(cmd)}")
    
    # if error in blender, regenerate the code with error message and max 3 attempts
    for attempt in range(3):
        try:
            subprocess.run(cmd, check=True)
            logger.success(f"Blender rendering completed: {output_path}")
            break
        except subprocess.CalledProcessError as e:
            logger.error(f"Blender rendering failed: {e}")
            if attempt < 2:
                logger.info(f"Regenerating Blender code due to error: {e}")
                blender_code = llm_to_scene(prompt)
                save_blender_code(blender_code)
                continue
            else:
                logger.error(f"Blender rendering failed after 3 attempts: {e}")
                raise
        except Exception as e:
            logger.error(f"Blender rendering failed: {e}")
            raise
    
    
    try:
        logger.info(f"Uploading {output_path} to Cloudinary...")
        
        upload_result = cloudinary.uploader.upload(
            str(output_path),
            folder=cloudinary_folder,
            public_id=f"{output_path.stem}_{int(time.time())}",
            overwrite=True,
            tags=["blender", "render", "image"],
        )
        
        logger.success(f"Upload successfull {upload_result['public_id']} !!")

        return upload_result
        
    except Exception as e:
        logger.error(f"Cloudinary upload failed: {e}")
        raise

## 6  Vision‑language verifier (stub)

In [140]:
def verify_scene(prompt: str, image_path: str) -> float:
    """Return fraction of relations judged correct by GPT-4V (stub)."""

    logger.info(f"Verifying scene ...") 

    system_message = 'You are an expert in evaluating PIL image objects, which contain rooms generated in Blender, and assigning them a score based on their spatial accuracy.'
    # prompt = f"Ascribe a decimal number ranging from 10-20 that scores the image in the following link {image_path} based on how well the generated scene satisfies the following JSON layout: {scene}"
    prompt += f"The image is at the following link: {image_path}. Just answer the question, no other text."

    response = OpenAPIClient.chat.completions.create(
            model='o4-mini',
            messages=[
                {'role':'system', 'content': system_message},
                {'role':'user', 'content': prompt}
            ],
    )

    return response.choices[0].message.content # assume perfect


## 7. Evaluation Part

In [141]:
def evaluation(prompt: str, url: str):
    verification_score_prompt = "Ascribe a decimal number ranging from 0-10 that scores the image in the following link {url} based on how well the generated scene satisfies the following JSON layout: {scene}"
    score = verify_scene(verification_score_prompt, url)
    result['test_questions']['question_1'] = {
        'question': verification_score_prompt,
        'answer': score
    }

    for i in range(3):
        multi_relation_prompt = f'''I want to assess the spatial accuracy of a scene created by a user prompt {prompt}. Write a yes/no question that will assess the multi-relation accuracy of the scene.
        Rules: multi-relations queries are queries where that ask about the spatial relationships of the objects/furniture that must appear given the prompt. Only return the question, no other text.'''
        question = OpenAPIClient.chat.completions.create(
            model='o4-mini',
            messages=[
                {'role':'system', 'content': 'You are an expert in asking questions on 3D scenes based on their spatial relationships.'},
                {'role':'user', 'content': multi_relation_prompt}
            ],
        )
        answer = verify_scene(question.choices[0].message.content, url)
        result['test_questions'][f'question_{i+2}']['question'] = question.choices[0].message.content
        result['test_questions'][f'question_{i+2}']['answer'] = answer

    return score

In [142]:
def save_blender_code(code, filename="blender_scene.py"):
    """Save the generated Blender Python code to a file"""
    logger.info(f"Saving Blender Python code to {filename} ...")
    # Create directory if it doesn't exist
    os.makedirs("blender_scripts", exist_ok=True)
    
    # Full path to save the file
    filepath = os.path.join("blender_scripts", filename)
    
    # Write the code to file
    with open(filepath, "w") as f:
        f.write(code)
    
    logger.success(f"Blender Python code saved to: {filepath}")

    return filepath

## 7  End‑to‑end pipeline

In [143]:
def plan_and_render(prompt: str):
    """
    Direct pipeline from prompt to render.
    
    Args:
        prompt: Initial prompt for scene generation
    """
    logger.info(f"Processing prompt and rendering scene...")
    
    try:
        # Generate scene from prompt
        logger.info("Generating scene from prompt ...")
        blender_code = llm_to_scene(prompt)
        print(blender_code)
        logger.success('Scene generated successfully.')

        # save the blender code
        save_blender_code(blender_code)

        # render the scene
        rendered_scene = render_and_upload_image(pyfile="assets/blender/script.py", 
                                               image_outfile="assets/renders/scene.png",    prompt=prompt)
        if not rendered_scene:
            raise ValueError("Render function returned empty result")
            
        # Evaluate scene
        logger.info("Evaluating scene...")
        evaluation(prompt, rendered_scene['secure_url'])
        logger.success('Scene evaluation completed.')
        
        logger.success(f"\nSuccess! Scene rendered successfully.")
        return rendered_scene
        
    except Exception as e:
        logger.error(f"Error in pipeline: {str(e)}")
        return None

## 8  Example run

In [144]:
example_prompt1 = "Design a bedroom (4.0 * 3.2 * 3.0 m), and put the bed snug against the left wall. Stand the wardrobe centred on the back wall. Rotate the desk so its front edge meets the foot of the bed, and rest a bedside lamp on the desk’s left corner. Position furniture logically with proper spacing and accessibility."
example_prompt2 = "Create a living room (4.5 * 3.5 * 3.0 m) Line the sofa along the left wall, place the coffee table directly in front of it, and fix the TV console against the right wall facing the sofa. A floor lamp should stand behind the sofa’s right arm."
example_prompt3 = "Design a dining room (3.6 × 3.6 × 3.0 m). Place the dining table in the middle of the room with the six chairs arranged around it; push the sideboard against the back wall behind the table, and hang the pendant light above the table."
example_prompt4 = "Design Bedroom (4.0 × 3.2 × 3.0 m). Place the bed centred on the back wall. Rotate the desk perpendicular to the bed and push it against the right wall where it meets the bed’s foot. Stand a floor lamp on the left side of the bed."


rendered_scene = plan_and_render(example_prompt1)
# result['prompt'] = example_prompt4
# result['image_url'] = rendered_scene['secure_url']


2025-06-03 23:30:05.578 | INFO     | __main__:plan_and_render:8 - Processing prompt and rendering scene...
2025-06-03 23:30:05.579 | INFO     | __main__:plan_and_render:12 - Generating scene from prompt ...
2025-06-03 23:30:20.695 | INFO     | __main__:llm_to_scene:25 - blender code generated successfully ...
2025-06-03 23:30:20.696 | SUCCESS  | __main__:plan_and_render:15 - Scene generated successfully.
2025-06-03 23:30:20.697 | INFO     | __main__:save_blender_code:3 - Saving Blender Python code to blender_scene.py ...
2025-06-03 23:30:20.698 | SUCCESS  | __main__:save_blender_code:14 - Blender Python code saved to: blender_scripts/blender_scene.py
2025-06-03 23:30:20.699 | INFO     | __main__:render_and_upload_image:20 - Rendering and uploading image to Cloudinary ...
2025-06-03 23:30:20.700 | INFO     | __main__:render_and_upload_image:27 - Running Blender: /Applications/Blender.app/Contents/MacOS/Blender -b --python assets/blender/script.py


```python
import bpy

# Function for creating objects
def create_obj(name, dimensions):
    bpy.ops.mesh.primitive_cube_add(
        size=1,
        enter_editmode=False,
        align='WORLD',
        location=(0, 0, dimensions[2]/2)
    )
    obj = bpy.context.object
    obj.dimensions = dimensions
    obj.name = name
    return obj

# Create room
room = create_obj('Room', (4.0, 3.2, 3.0))

# Create bed
bed = create_obj('Bed', (1.9, 0.9, 0.6))
bed.location.x = 0 # Centered on the back wall
bed.location.y = -((room.dimensions.y - bed.dimensions.y) / 2) # Pushed against the back wall

# Create desk
desk = create_obj('Desk', (1.2, 0.6, 0.75))
desk.rotation_euler.z = 1.5708 # Perpendicular to the bed
desk.location.x = bed.location.x + bed.dimensions.x /2 + desk.dimensions.y / 2 # Meets the bed foot
desk.location.y = room.dimensions.y / 2 - desk.dimensions.x / 2 # Pushed against the right wall

# Create floor lamp
lamp = create_obj('Lamp', (0.3, 0.3, 1.7))
lamp.location.x = -room.dimensio

Traceback (most recent call last):
  File "/Users/divinirakiza/Workspaces/CALTECH/cs159/snapTo3D/3D_craft/assets/blender/script.py", line 86, in <module>
    bsdf.inputs['Roughness'].default_value = None
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: bpy_struct: item.attr = val: NodeSocketFloatFactor.default_value expected a float type, not NoneType
2025-06-03 23:30:21.386 | SUCCESS  | __main__:render_and_upload_image:33 - Blender rendering completed: assets/renders/scene.png
2025-06-03 23:30:21.386 | INFO     | __main__:render_and_upload_image:51 - Uploading assets/renders/scene.png to Cloudinary...


Blender 4.4.1 (hash d8845b3bb572 built 2025-04-15 01:30:48)

Blender quit


2025-06-03 23:30:22.121 | SUCCESS  | __main__:render_and_upload_image:61 - Upload successfull one_step_renders/scene_1749018621 !!
2025-06-03 23:30:22.122 | INFO     | __main__:plan_and_render:27 - Evaluating scene...
2025-06-03 23:30:22.122 | INFO     | __main__:verify_scene:4 - Verifying scene ...
2025-06-03 23:30:27.245 | INFO     | __main__:verify_scene:4 - Verifying scene ...
2025-06-03 23:30:36.025 | INFO     | __main__:verify_scene:4 - Verifying scene ...
2025-06-03 23:30:44.859 | INFO     | __main__:verify_scene:4 - Verifying scene ...
2025-06-03 23:30:54.243 | SUCCESS  | __main__:plan_and_render:29 - Scene evaluation completed.
2025-06-03 23:30:54.244 | SUCCESS  | __main__:plan_and_render:31 - 
Success! Scene rendered successfully.


In [145]:
result

{'id': 1,
 'test_type': 'multi-relation',
 'prompt': '',
 'image_url': '',
 'test_questions': {'question_1': {'question': 'Ascribe a decimal number ranging from 0-10 that scores the image in the following link {url} based on how well the generated scene satisfies the following JSON layout: {scene}',
   'answer': '8.3'},
  'question_2': {'question': 'Is the desk perpendicular to the bed and pushed against the right wall at the bed’s foot, with a floor lamp standing on the left side of the bed?',
   'answer': 'No.'},
  'question_3': {'question': 'Is the bed centered on the back wall with the desk placed perpendicular to it and flush against the right wall at its foot, while the floor lamp stands on the left side of the bed?',
   'answer': 'Yes.'},
  'question_4': {'question': 'Is the bed centered on the back wall, the desk perpendicular to the bed and pushed against the right wall at the foot of the bed, and the floor lamp standing on the left side of the bed?',
   'answer': 'No.'}}}

In [146]:

result

{'id': 1,
 'test_type': 'multi-relation',
 'prompt': '',
 'image_url': '',
 'test_questions': {'question_1': {'question': 'Ascribe a decimal number ranging from 0-10 that scores the image in the following link {url} based on how well the generated scene satisfies the following JSON layout: {scene}',
   'answer': '8.3'},
  'question_2': {'question': 'Is the desk perpendicular to the bed and pushed against the right wall at the bed’s foot, with a floor lamp standing on the left side of the bed?',
   'answer': 'No.'},
  'question_3': {'question': 'Is the bed centered on the back wall with the desk placed perpendicular to it and flush against the right wall at its foot, while the floor lamp stands on the left side of the bed?',
   'answer': 'Yes.'},
  'question_4': {'question': 'Is the bed centered on the back wall, the desk perpendicular to the bed and pushed against the right wall at the foot of the bed, and the floor lamp standing on the left side of the bed?',
   'answer': 'No.'}}}

In [147]:
results = {"results": []}

with open('tests/one_step_results.json', 'r') as f:
    
    results = json.load(f)
print('here')
results["results"].append(result)

with open('tests/one_step_results.json', 'w') as f:
    json.dump(results, f, indent=4)

FileNotFoundError: [Errno 2] No such file or directory: 'tests/one_step_results.json'